In [1]:
import sklearn
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import credenciais.credenciais_pedro_report as cpr
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
from datetime import datetime as dt
import datetime as datetime
import warnings
warnings.filterwarnings('ignore')

df= pd.read_csv('df_no_duplicates.csv')
df_n_dup = df.drop_duplicates().reset_index(drop=True)

In [2]:
# Primeiro bloco de código. Definição de o que é um Outlier.
# Se quiserem adicionar ou criar algum outro critério, é só mudar esse bloco, acho que é o mais importante...
# Vale a pena dar uma checada pra ver se esses valores realmente tão fazendo sentido pra chamar de outliers...
# Dessa vez mais na prática

l = []
for i in df_n_dup['category_nm'].unique():
    if df_n_dup[(df_n_dup["category_nm"].isin([i]))&(df_n_dup["vendido"] == 1)].dropna().shape[0] > 50:
        collectn_1 = np.array(df_n_dup[(df_n_dup['category_nm'].isin([i]))&(df_n_dup['vendido'] == 1)].dropna()['2_1'])
        collectn_2 = np.array(df_n_dup[(df_n_dup['category_nm'].isin([i]))&(df_n_dup['vendido'] == 1)].dropna()['3_1'])
        collectn_3 = np.array(df_n_dup[(df_n_dup['category_nm'].isin([i]))&(df_n_dup['vendido'] == 1)].dropna()['1_1'])
        collectn_4 = np.array(df_n_dup[(df_n_dup['category_nm'].isin([i]))&(df_n_dup['vendido'] == 1)].dropna()['densidade'])  
        collectn_5 = np.array(df_n_dup[(df_n_dup['category_nm'].isin([i]))&(df_n_dup['vendido'] == 1)].dropna()['gross_weight_nu'])            

        l.append([i,np.median(collectn_1),np.median(collectn_2),                      
                  np.std(collectn_1),np.std(collectn_2),
                  np.median(collectn_3),np.mean(collectn_3),
                  np.std(collectn_3),
                  (np.median(collectn_1) - np.std(collectn_1)*1.645, np.median(collectn_1) + np.std(collectn_1)*1.645),  
                  (np.median(collectn_2) - np.std(collectn_2)*1.645, np.median(collectn_2) + np.std(collectn_2)*1.645),
                  (np.median(collectn_3) - np.std(collectn_3)*1.645, np.median(collectn_3) + np.std(collectn_3)*1.645),
                  (np.median(collectn_4) - np.std(collectn_4)*1.645, np.median(collectn_4) + np.std(collectn_4)*1.645),
                  (np.median(collectn_5) - np.std(collectn_5)*1.645, np.median(collectn_5) + np.std(collectn_5)*1.645),
                  len(collectn_3)])
df_intervals = pd.DataFrame(l,columns=['Segment','1st','2nd','2nd_std','cfd_int1','Max_Median','Max_Mean','Max_SD','2_1','3_1','1_1','densidade','gross_weight_nu','Sample'])\
            .dropna().sort_values(by=['1st','2nd'],ascending=False).reset_index(drop=True)

In [3]:
# Junção dos Outliers encontrados com base nos critérios definidos Acima. 
# Adição de uma coluna nova representando eles.


dictionary = {'2_1':[],'3_1':[],'1_1':[],'densidade':[],'gross_weight_nu':[]}
print('Juntando Outliers!')
for i,v in zip(df_n_dup.iloc,tqdm(range(len(df_n_dup)))):
    if list(df_intervals[df_intervals['Segment'] == i['category_nm']]['3_1']) != []:
        for j in dictionary.keys():
            dictionary[j].append(df_intervals[df_intervals['Segment'] == i['category_nm']][j].values[0][0] < i[j] <df_intervals[df_intervals['Segment'] == i['category_nm']][j].values[0][1])    
    else:
        for j in dictionary.keys():
            dictionary[j].append('Pouca Amostra')
for i in dictionary.keys():
    df_n_dup[f'{i}_int'] = dictionary[i]

Juntando Outliers!


100%|█████████████████████████████████▉| 292390/292391 [05:44<00:00, 849.61it/s]


In [23]:
print('-'*15,'Iniciando 4 CVs x 3 Modelos x 15 Combinações HiperParâmetros x 102 Categorias', '-'*15)

# Fase 1 --> criar todas as combinações possíveis de regressores com seus respectivos hiperparâmetros

modelos = {'Random Forest Regressor':{},
          'Gradient Boosting Regressor':{},
          'MLP Regressor': {}}

for n_estimatorRF, n_estimatorGB, hidden_layers in zip([75,85,100,115,125],[75,85,100,115,125],[75,85,100,125,150]):
    for min_sample_splitRF,min_sample_splitGB,learning_rate in zip([2,4,8],[2,4,8],[0.00075,0.001,0.002]):
        modelos['Random Forest Regressor']\
        .update({f'params_1: {n_estimatorRF} - params_2: {min_sample_splitRF}':
                RandomForestRegressor(n_estimators=n_estimatorRF,min_samples_split=min_sample_splitRF)})
        modelos['Gradient Boosting Regressor']\
        .update({f'params_1: {n_estimatorGB} - params_2: {min_sample_splitGB}':
                GradientBoostingRegressor(n_estimators=n_estimatorGB,min_samples_split=min_sample_splitGB)})        
        modelos['MLP Regressor']\
        .update({f'params_1: {hidden_layers} - params_2: {learning_rate}':
                MLPRegressor(hidden_layer_sizes=hidden_layers,learning_rate_init=learning_rate)})        

total = pd.DataFrame({'MAE':[],'MSE':[],'MAPE':[],'Amostra':[]})

# Primeiro Loop, verificando categoria por categoria

for i,v in zip(df_n_dup[df_n_dup['2_1_int'] != 'Pouca Amostra']['category_nm'].unique(),tqdm(range(len(df_n_dup[df_n_dup['2_1_int'] != 'Pouca Amostra']['category_nm'].unique())))):
    try:
        df_testing = df_n_dup[(df_n_dup['category_nm'] == i)&
                             (df_n_dup['gross_weight_nu_int'] == 1)&(df_n_dup['3_1_int'] == 1)&
                             (df_n_dup['2_1_int'] == 1)&(df_n_dup['1_1_int'] == 1)&(df_n_dup['densidade_int'] == 1)
                            ]

        df_modelo = df_testing[['brand_nm','3_1','2_1','1_1','volume','gross_weight_nu']]
        df_modelo = pd.concat([df_modelo.drop(columns='brand_nm'), pd.get_dummies(df_modelo['brand_nm'])],1)


        dictionary_models = {}
        partitions = KFold(n_splits=4)
        
        #Terceiro Loop, verificando Cross-Validation por Cross-Validation

        for train,test in partitions.split(df_modelo):
            X_test, y_test = df_modelo.iloc[test].drop(columns='gross_weight_nu'), df_modelo.iloc[test][['gross_weight_nu']]
            X_train, y_train = df_modelo.iloc[train].drop(columns='gross_weight_nu'), df_modelo.iloc[train][['gross_weight_nu']]
            dictionary_models[f'{test[0]} - {test[-1]}'] = {}
            
            # Quarto e Quinto Loop, Verificando Modelo por Modelo
            
            for modelo in modelos.keys():
                for hiperparametros in modelos[modelo].keys():

                    predicting = modelos[modelo][hiperparametros].fit(X_train,y_train)
                    predictions = predicting.predict(X_test)

                    dictionary_models[f'{test[0]} - {test[-1]}'].update({f'{modelo} - {hiperparametros}':{
                      'MAE':mean_absolute_error(predictions,y_test),
                      'MSE':mean_squared_error(predictions,y_test),
                      'MAPE':mean_absolute_percentage_error(predictions,y_test),
                      'Amostra':len(y_test)}})
        
        # Só colocando tudo em um formato mais "visual", ou seja, em um DataFrame
        metricas ={'MAE':{},'MSE':{},'MAPE':{},'Amostra':{}}
        for item,value in metricas.items():
            models = {}
            for j in dictionary_models[list(dictionary_models.keys())[0]].keys():
                mean = []
                for k in dictionary_models.keys():
                    mean.append(dictionary_models[k][j][item])
                models.update({j:np.mean(mean)})
            metricas[item].update(models)
        resultados_tuning = pd.DataFrame(metricas)
        
        # Adicionando os hiperparâmetros mais eficientes encontrados para cada categoria
        
        dictionary = {'params_1':[],'params_2':[]}
        for k in resultados_tuning.index:
            dictionary['params_1'].append(float(k.split(' - ')[1].split(': ')[1]))
            dictionary['params_2'].append(float(k.split(' - ')[2].split(': ')[1]))
        for k in dictionary.keys():
            resultados_tuning[k] = dictionary[k]
        
        # 
        df_melhor = pd.DataFrame(resultados_tuning.sort_values(by=['MAPE','MAE']).iloc[0])
        df_melhor.loc['Modelo'] = df_melhor.columns[0].split(' - ')[0]
        total = pd.concat([total,df_melhor.rename(columns={df_melhor.columns[0]:i}).transpose()])
    except: 
        pass
total = total.rename_axis('Categoria').reset_index()    

display(total)

--------------- Iniciando 4 CVs x 3 Modelos x 15 Combinações HiperParâmetros x 102 Categorias ---------------


 99%|████████████████████████████████████████▌| 101/102 [21:03<00:12, 12.51s/it]


,Categoria,MAE,MSE,MAPE,Amostra,params_1,params_2,Modelo
0,"Livros, Revistas e Comics > Outros",309.990466,332197.707534,0.738586,285.25,75.0,2.0,Gradient Boosting Regressor
1,Brinquedos e Hobbies > Brinquedos de Montar > ...,209.393342,102295.094069,0.399364,400.0,125.0,8.0,Random Forest Regressor
2,"Casa, Móveis e Decoração > Têxteis de Casa e D...",656.646123,753221.072067,0.513574,39.5,100.0,2.0,Random Forest Regressor
3,"Casa, Móveis e Decoração > Organização para Casa",370.200281,240531.211324,0.60245,385.0,115.0,8.0,Gradient Boosting Regressor
4,"Casa, Móveis e Decoração > Cozinha > Louça e A...",189.563204,83797.659298,0.440932,227.0,75.0,8.0,Gradient Boosting Regressor
5,Informática > Conectividade e Redes > Roteadores,186.788996,74674.998129,0.489468,84.75,115.0,4.0,Random Forest Regressor
6,Construção > Outros,311.26604,184411.298245,0.68729,465.5,125.0,8.0,Gradient Boosting Regressor
7,"Arte, Papelaria e Armarinho > Outros",195.508548,82970.003332,0.646147,173.5,75.0,4.0,Gradient Boosting Regressor
8,Construção > Mobiliário para Banheiros > Torne...,287.749746,172332.062706,0.378094,117.25,115.0,2.0,Random Forest Regressor


In [25]:
# Agora a gente vai usar o melhor modelo encontrado pra cada categoria.
# A gente descobre isso no dataframe que a gente criou lá atrás...

from tqdm import tqdm

FixFinalDF = pd.DataFrame({'sku_nm':[],'brand_nm':[],'3_1':[],'2_1':[],'1_1':[],'volume':[],'gross_weight_nu':[]})
print('Juntando Estimativas!!')
for i,v in zip(df_n_dup[df_n_dup['2_1_int'] != 'Pouca Amostra']['category_nm'].unique(),tqdm(range(len(df_n_dup[df_n_dup['2_1_int'] != 'Pouca Amostra']['category_nm'].unique())))):
    try:
        df_train = df_n_dup[(df_n_dup['category_nm'] == i)&
                             (df_n_dup['gross_weight_nu_int'] == 1)&(df_n_dup['3_1_int'] == 1)&
                             (df_n_dup['2_1_int'] == 1)&(df_n_dup['1_1_int'] == 1)&(df_n_dup['densidade_int'] == 1)
                            ][
                            ['brand_nm','3_1','2_1','1_1','volume','gross_weight_nu']]
        dummies = pd.get_dummies(df_train['brand_nm'])
        df_train = pd.concat([df_train.drop(columns='brand_nm'), dummies],1)

        df_fixing = df_n_dup[(df_n_dup['category_nm'] == i)&
                             (df_n_dup['gross_weight_nu_int'] == 0)&(df_n_dup['3_1_int'] == 1)&
                             (df_n_dup['2_1_int'] == 1)&(df_n_dup['1_1_int'] == 1)&(df_n_dup['densidade_int'] == 1)
                            ][
                            ['sku_nm','brand_nm','3_1','2_1','1_1','volume','gross_weight_nu']]
        
        parametros = total[total['Categoria'] == i][['Modelo','params_1','params_2']]
        
        for i in dummies.columns:
            l = []
            for j in df_fixing['brand_nm']:
                if j == i:
                    l.append(1)
                else:
                    l.append(0)
            df_fixing[i] = l
        
        
        if parametros['Modelo'].values[0] == 'Random Forest Regressor':
            predicting_final = RandomForestRegressor(n_estimators= int(parametros['params_1'].values[0]),
                                                     min_samples_split = int(parametros['params_2'].values[0]))\
            .fit(df_train.drop(columns='gross_weight_nu'),df_train['gross_weight_nu'])
        elif parametros['Modelo'].values[0] == 'Gradient Boosting Regressor':
            predicting_final = GradientBoostingRegressor(n_estimators= int(parametros['params_1'].values[0]),
                                                     min_samples_split = int(parametros['params_2'].values[0]))\
            .fit(df_train.drop(columns='gross_weight_nu'),df_train['gross_weight_nu'])
        elif parametros['Modelo'].values[0] == 'MLP Regressor':
            predicting_final = MLPRegressor(hidden_layer_sizes = parametros['params_1'].values[0],
                                                          learning_rate_init = parametros['params_2'].values[0])\
                .fit(df_train.drop(columns='gross_weight_nu'),df_train['gross_weight_nu'])        
        
        predictions = predicting_final.predict(df_fixing.drop(columns=['gross_weight_nu','brand_nm','sku_nm']))

        FixFinalDF = pd.concat([FixFinalDF,pd.concat([df_fixing[['sku_nm','brand_nm','3_1','2_1','1_1','volume','gross_weight_nu']].reset_index(drop=True)
                   ,pd.DataFrame(predictions).rename(columns={0:'Fix'})],1)])
    except:
        pass
FixFinalDF

Juntando Estimativas!!


 99%|████████████████████████████████████████▌| 101/102 [00:39<00:00,  2.55it/s]


,sku_nm,brand_nm,3_1,2_1,1_1,volume,gross_weight_nu,Fix
0,Bíblia Do Obreiro Com Letras Vermelhas Ra - Lu...,CLARO,0.285714,0.666667,6.0,1764.0,76000.0,563.138890
1,Livro - Direito De Família - Madaleno,DIREITO,0.166667,0.708333,4.0,1632.0,209000.0,601.974712
2,Bíblia Da Mulher Vitoriosa Média - Letra Gigan...,VINHO,0.181818,0.636364,4.0,1232.0,85000.0,554.567819
0,Lego Creator - Motores De Corrida Radical - 31072,Lego,0.250000,0.700000,5.0,1400.0,3000.0,246.402504
1,LEGO Friends - Carrinho de Sorvetes - Stephani...,LEGO,0.357143,0.857143,5.0,840.0,3000.0,94.345366
...,...,...,...,...,...,...,...,...
100,Papel Vegetal Filiperson A4 60G 10 Folhas Tran...,Filiperson,0.071429,0.750000,2.0,1176.0,3000.0,275.739982
101,Lapis De Cor 36 Cores Faber Multicolor Super.,FABER-CASTELL,0.100000,0.700000,2.0,560.0,3000.0,245.050532
102,Verniz Vitral Acrilex Preto,ACRILEX,0.500000,0.777778,9.0,2268.0,3000.0,214.499705
103,Tinta Acrilica Profissional Amarelo Pele 20ml ...,Acrilex,0.176471,0.647059,3.0,561.0,3000.0,241.021034
